This notebook demonstrates how to create a spatialite database from GMR inversions. Spatialite was chosen as the format as it is compact, easy to set up and readable by various software including GIS and python. Much of the creation depends on two spreadsheets referenced in this notebok which contain metadata, key spatial information and map inversion directories.

Neil Symington
neil.symington@gs.gov.au

In [1]:
import shapely.wkb
import shapely.wkt
import os
import pandas as pd
# sqlite/spatialite
from sqlalchemy import create_engine, event, ForeignKey
from sqlalchemy import Column, Integer, String, Float, Date, Boolean
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlite3 import dbapi2 as sqlite
from scipy.io import loadmat
import sqlite3
import numpy as np

In [2]:
# Configuration for local environment

user = 'Neil Symington'
#user = 'Alex Ip'

if user == 'Neil Symington':
    # Neil Symington's local configuration
    DB_ROOT = r"C:\GA\SNMR"
    SPATIALITE_PATH = r'C:\Users\symin\mod_spatialite-NG-win-amd64\mod_spatialite-NG-win-amd64'
elif user == 'Alex Ip':
    # Alex Ip's local configuration
    DB_ROOT = r"F:\Groundwater\SNMR" # Alex's external hard drive
    SPATIALITE_PATH = None # Not required - already in path

SyntaxError: invalid syntax (<ipython-input-2-d12729fbf395>, line 10)

In [ ]:
# load spatialite extension for sqlite if required. 
# Make sure that mod_spatialite.dll is located in a folder that is in your system path
# This will only work on windows computers

if SPATIALITE_PATH:
    os.environ['PATH'] = SPATIALITE_PATH + ';' + os.environ['PATH']

In [ ]:
DB_PATH = os.path.join(DB_ROOT, r"East_Kimberley_SNMR.sqlite")

if os.path.exists(DB_PATH):
        os.remove(DB_PATH)

        
engine = create_engine('sqlite:///' + DB_PATH, module=sqlite, echo=False)

@event.listens_for(engine, 'connect')
def connect(dbapi_connection, connection_rec):
    dbapi_connection.enable_load_extension(True)
    dbapi_connection.execute('SELECT load_extension("mod_spatialite")')

# create spatialite metadata
print('creating spatial metadata...')
engine.execute("SELECT InitSpatialMetaData(1);")

In [ ]:
Base = declarative_base()

class Sites(Base):
    __tablename__ = 'sites'
    site_id = Column(Integer, index=True, primary_key=True)
    field_id = Column("Field_ID", String(20))
    site_code = Column("Site_Code", String(40))
    mid_x = Column("mid_X", Float)
    mid_y = Column("mid_Y", Float)
    declination = Column("declination_angle", Float)
    inclination = Column("inclination_angle", Float)
    date = Column("date", Date)
    loop_width = Column("loop_width", Float)
    coil_type = Column("coil_type", String(20))
    geometry = Column(String)
      
    
class Acquisitions(Base):
    __tablename__ = 'acquisitions'
    acquisition_id = Column(Integer, index=True, primary_key=True)
    pulse_sequence = Column("pulse_sequence", String(5))
    pulse_length = Column("pulse_length", Float)
    
    
    site_id = Column(Integer, ForeignKey('sites.site_id'))
    sites = relationship("Sites")

    
class Inverse_model_metadata(Base):
    __tablename__ = 'inverse_model_metadata'
    inversion_id = Column(Integer, index=True, primary_key=True)
    doi = Column('Depth_of_Investigation', Float)
    reg_factor = Column('reg_factor', Float)
    cond_profile = Column("conductive_earth_model", Boolean)
    inversion_software = Column("Inversion_software", String(50))
    
    
    acquisition_id = Column(Integer, ForeignKey('acquisitions.acquisition_id'))
    acquisitions = relationship("Acquisitions")
    
class Inverse_models(Base):
    __tablename__ = 'inverse_models'
    table_id = Column(Integer, index=True, primary_key=True)
    depth_from = Column('Depth_from', Float)
    depth_to = Column('Depth_to', Float)
    mobile_water_content = Column("Mobile_water_content", Float)
    bound_water_content = Column("Bound_water_content", Float)
    total_water_content = Column("Total_water_content", Float)
    T1 = Column("T1", Float)
    T2 = Column("T2", Float)
    T2_star = Column("T2*", Float)
    frequency = Column("frequency", Float)
    phase = Column("phase", Float)
    
    site_id = Column(Integer, ForeignKey('sites.site_id'))
    sites = relationship("Sites")
    
    acquisition_id = Column(Integer, ForeignKey('acquisitions.acquisition_id'))
    acquisitions = relationship("Acquisitions")

    
    inversion_id = Column(Integer, ForeignKey('inverse_model_metadata.inversion_id'))
    inversions = relationship("Inverse_model_metadata")



In [ ]:
Sites.__table__

In [ ]:
Base.metadata.create_all(engine)

In [ ]:
infile = os.path.join(DB_ROOT, r"EK_SNMR_location_metadata.csv")

df = pd.read_csv(infile)

df['date'] =pd.to_datetime(df.date, format ="%d/%m/%Y")

df = df.sort_values(by='date').reset_index(drop=True)

df

In [ ]:
# Drop columns that are not needed in the table


df.rename(columns = {'X': 'mid_X', 'Y': 'mid_Y', 'bo_inc_angle_deg': 'inclination_angle',
                     'bo_dec_angle_deg': 'declination_angle'}, inplace = True)

In [ ]:
# Bring in the data into a dataframe

infile = os.path.join(DB_ROOT, r"EK_conductive_earth_inversion_spreadsheet.csv")
df_acquisition = pd.read_csv(infile)

df_acquisition.columns

In [ ]:
# Now add the location index and find it using a join on the site id column

loc_id = -999 * np.ones(len(df_acquisition['Field_ID']),
                      dtype = np.int64)

for i , item in enumerate(df_acquisition['Field_ID'].values):
    # find the location index
    loc_id[i] = df[df['Field_ID'] == item].index[0]


df_acquisition = df_acquisition.assign(site_id = loc_id)

df_acquisition['pulse_length'].dtype

df_acquisition = df_acquisition.sort_values(by='site_id').reset_index(drop=True)

df_acquisition.columns

In [ ]:

df_acquisition['inversion_file'] = ''

for index, row in df_acquisition.iterrows():
    new_path = row['save_pathname'].replace(r'D:\EastKimberley_SNMR\SNMR_database', DB_ROOT)
    
    df_acquisition['inversion_file'].iloc[index] = new_path + row['save_filename'] + '\\' + row['save_filename'] + '_1d_inversion.txt'



In [ ]:
df_acquisition['inversion_file'].iloc[0]

In [ ]:
# Define headers for the various inversion files

header_dict  = {}

header_dict['FID'] = ['Depth_from', 'Depth_to', 'K_rel', 'water_content', 'T2*', 'frequency',
          'phase', 'T1', 'Bound_water_content', 'Mobile_water_content', 'Total_water_content']

header_dict['CPMG'] = ['Depth_from', 'Depth_to', 'K_rel', 'Mobile_water_content', 'T2']

header_dict['T1'] = ['Depth_from', 'Depth_to', 'K_rel', 'water_content', 'T2*', 'frequency',
          'phase', 'T1', 'Bound_water_content', 'Mobile_water_content', 'Total_water_content']

# This function parses the text file automatically created by the inversion algorithm


def parse_inversion_file(infile, pulse_sequence):
    
    '''
    infile: the filename of the text file
    sequence: ['FID', 'CPMG', 'T1']
    '''
    # Get header based on the sequence
    
    header = header_dict[pulse_sequence]
    
    #import data into numpy array
    data = np.loadtxt(infile, usecols = np.arange(0,len(header)))
    
    # Check if final row is just zeros
    if data[-1,0] == 0.:
        data = data[:-1,:]
    
    # Create a pandas dataframe
    df_inv = pd.DataFrame(data, columns = header)
    
    # Add a null column of T2* (FID and T1) or T2 (CPMG)
    #if 'T2*' not in header:
    #    df_inv['T2*'] = np.nan*np.ones(df_inv.shape[0])
    #elif 'T2' not in header:
    #    df_inv['T2'] = np.nan*np.ones(df_inv.shape[0])
    
    
    return df_inv

In [ ]:
# Function to get the index of acquisition entry given site, and  acquisition parameters

def get_foreign_keys(df_acquisition, Field_ID, pulse_sequence, pulse_length):
    # Define criterion    
    criterion1 = df_acquisition['Field_ID'].map(lambda x: x== Field_ID)
    criterion2 = df_acquisition['pulse_sequence'].map(lambda x: x== pulse_sequence)
    criterion3 = df_acquisition['pulse_length'].map(lambda x: x == pulse_length)

    acquisition = df_acquisition[criterion1][criterion2][criterion3]
    
    return acquisition.site_id, acquisition.index

# Function for updating the acquisition metadata dataframe

def update_inversion_metadata(inv_id, acqu_ind, doi, 
                              reg_parameter, cond_profile):
    # First we convert the cond profile to a boolean
    
    if cond_profile == 0:
        conductive_earth = False
    else:
        conductive_earth = True
    
    d = {'inversion_id': inv_id,
         'acquisition_id': acqu_ind, 
         'DOI': doi,
         'conductive_earth': conductive_earth,
         'regularisation_parameter': reg_parameter}

    # Now add an entry to the df_invmet dataframe

    df_temp = df_invmet.append(pd.DataFrame.from_dict([d]),
                                ignore_index = True)
    return df_temp
    

In [ ]:
# Create a table for inversion metadata

df_invmet = pd.DataFrame(columns =  ['inversion_id', 'acquisition_id', 'DOI',
                                     'conductive_earth', 'regularisation_parameter'])

In [ ]:
# Create an empty dataframe into which to add data

df_inversion = pd.DataFrame(columns = ['inversion_id', 'Depth_from', 'Depth_to', 'K_rel', 
                                       'water_content', 'T2*','T2', 'frequency', 'phase', 'T1',
                                       'Bound_water_content', 'Mobile_water_content', 
                                       'Total_water_content', 'acquisition_id', 'site_id'])
inv_id = 0

# Now we import the inversion data

for index, row in df_acquisition.iterrows():
    
    infile = row['inversion_file']
    
    try:
        df_inv = parse_inversion_file(infile, row.pulse_sequence)
    
        site_ind, acqu_ind = row['site_id'], index
    
        reg_parameter = row['reg_factor']
        cond_profile = row['use_cond_profile']
    
    
        ##TODO fix this
        # Extract the depth of investigation from the matlab header
        fig = loadmat('\\'.join(infile.split('\\')[:-1]) + '\\resmatrix.fig')
    
        # Rather ugly hack to extract the depth of investigation from a 
        # .fig file

        doi = fig['hgS_070000'][0][0][3][1][0][3][1][0][2][0][0][7][0][0]
    
        # Update the inversion metadata dataframe
                
        df_invmet = update_inversion_metadata(inv_id, acqu_ind,
                                          doi, reg_parameter, cond_profile)
    

        df_inv['acquisition_id'] = int(acqu_ind)
        df_inv['site_id'] = int(site_ind)
        df_inv['inversion_id'] = inv_id
        
        df_inversion = df_inversion.append(df_inv)
        inv_id += 1
        
    except OSError:
        
        print(row.Field_ID)

# Reset the inversion index so t can be used as a primary key
df_inversion.reset_index(inplace=True)



In [ ]:
df_invmet.set_index('inversion_id', inplace = True)

In [ ]:
# Now we try to get the dataframes into the database classes

all_sites = []

for index, row in df.iterrows():
    site = Sites(site_id = index,
                 field_id = row["Field_ID"], 
                site_code = row["Site_Code"], 
                mid_x = row["mid_X"], mid_y = row["mid_Y"],
                declination = row["declination_angle"],
                inclination = row["inclination_angle"],
                date = pd.to_datetime(row['date'], format = '%d/%m/%Y').date(),
                loop_width = row["loop_width"],
                coil_type = row["coil_type"],
                 geometry = row['geometry'])
    all_sites.append(site)


In [ ]:
# Now we try to get the dataframes into the database classes

all_acquisitions = []

for index, row in df_acquisition.iterrows():
    acquisition = Acquisitions(acquisition_id = index,
                               pulse_sequence = row['pulse_sequence'],
                               pulse_length = row['pulse_length'],
                               site_id = row['site_id'])
                               
    all_acquisitions.append(acquisition)


In [ ]:
all_inversion_metadata = []

for index, row in df_invmet.iterrows():
    inv_met = Inverse_model_metadata(doi = row['DOI'],
                                     reg_factor = row['regularisation_parameter'],
                                     cond_profile = True,
                                     inversion_software = 'GMRInversion1D_CLI_v2.7.3',
                                     acquisition_id = row['acquisition_id'],
                                     inversion_id = index)
    all_inversion_metadata.append(inv_met)
                                     

In [ ]:

all_inversions = []

for index, row in df_inversion.iterrows():
    inversion = Inverse_models(table_id = index,
                               depth_from = row['Depth_from'],
                           depth_to = row['Depth_to'],
                           mobile_water_content = row['Mobile_water_content'],
                           bound_water_content = row["Bound_water_content"],
                           total_water_content =row["Total_water_content"],
                           T1 = row["T1"],
                           T2 = row["T2"],
                           T2_star = row["T2*"],
                           frequency = row["frequency"],
                           phase = row["phase"],
                           site_id = row['site_id'],
                           acquisition_id = row['acquisition_id'],
                              inversion_id = row['inversion_id'])
                               
    all_inversions.append(inversion)


In [ ]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
session.add_all(all_sites)
session.add_all(all_acquisitions)
session.add_all(all_inversions)
session.add_all(all_inversion_metadata)

session.commit()

In [ ]:
# add a Spatialite geometry column called 'geom' to the table, using ESPG 28352,
# data type POLYGON and 2 dimensions (x, y)
engine.execute("SELECT AddGeometryColumn('sites', 'geom', 28352, 'POLYGON', 2);")

# update the yet empty geom column by parsing the well-known-binary objects from the geometry column into 
# Spatialite geometry objects
engine.execute("UPDATE sites SET geom=GeomFromText(geometry, 28352);")

